In [2]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True)

In [3]:
import pandas as pd
cancer_X = pd.DataFrame(data=data_breast_cancer['data'], columns=data_breast_cancer['feature_names'])
cancer_X = cancer_X[['mean texture', 'mean symmetry']]
cancer_y = pd.DataFrame(data=data_breast_cancer['target'], columns=['target'])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer_X, cancer_y, test_size=0.2, random_state=42)

In [5]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

dt_clf = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)
lr_clf = LogisticRegression(random_state=42).fit(X_train, y_train)
knn_clf = KNeighborsClassifier().fit(X_train, y_train)

hard_voting_clf = VotingClassifier(
    estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                ('lr', LogisticRegression(random_state=42)),
                ('knn', KNeighborsClassifier())],
    voting='hard')

soft_voting_clf = VotingClassifier(
    estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                ('lr', LogisticRegression(random_state=42)),
                ('knn', KNeighborsClassifier())],
    voting='soft')

/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [6]:
hard_voting_clf.fit(X_train, y_train)
soft_voting_clf.fit(X_train, y_train)

/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarnin

VotingClassifier(estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                             ('lr', LogisticRegression(random_state=42)),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [7]:
dt_predicted_train = dt_clf.predict(X_train)
dt_predicted_test = dt_clf.predict(X_test)

lr_predicted_train = lr_clf.predict(X_train)
lr_predicted_test = lr_clf.predict(X_test)

knn_predicted_train = knn_clf.predict(X_train)
knn_predicted_test = knn_clf.predict(X_test)

hard_voting_predicted_train = hard_voting_clf.predict(X_train)
hard_voting_predicted_test = hard_voting_clf.predict(X_test)

soft_voting_predicted_train = soft_voting_clf.predict(X_train)
soft_voting_predicted_test = soft_voting_clf.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
acc_list = []

acc_list.append((accuracy_score(y_train, dt_predicted_train), accuracy_score(y_test, dt_predicted_test)))
acc_list.append((accuracy_score(y_train, lr_predicted_train), accuracy_score(y_test, lr_predicted_test)))
acc_list.append((accuracy_score(y_train, knn_predicted_train), accuracy_score(y_test, knn_predicted_test)))
acc_list.append((accuracy_score(y_train, hard_voting_predicted_train), accuracy_score(y_test, hard_voting_predicted_test)))
acc_list.append((accuracy_score(y_train, soft_voting_predicted_train), accuracy_score(y_test, soft_voting_predicted_test)))
acc_list

[(1.0, 0.6140350877192983),
 (0.7230769230769231, 0.7017543859649122),
 (0.7714285714285715, 0.6403508771929824),
 (0.8351648351648352, 0.6929824561403509),
 (0.9648351648351648, 0.6666666666666666)]

In [9]:
import pickle
with open('acc_vote.pkl', 'wb') as acc_pickle:
    pickle.dump(acc_list, acc_pickle)

In [10]:
clf_list = []

clf_list.append(dt_clf)
clf_list.append(lr_clf)
clf_list.append(knn_clf)
clf_list.append(hard_voting_clf)
clf_list.append(soft_voting_clf)

In [11]:
with open('vote.pkl', 'wb') as vote_pickle:
    pickle.dump(clf_list, vote_pickle)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=30, max_samples=1.0, bootstrap=True, random_state=42).fit(X_train, y_train)
bag_50_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=30, max_samples=0.5, bootstrap=True, random_state=42).fit(X_train, y_train)
past_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=30, max_samples=1.0, bootstrap=False, random_state=42).fit(X_train, y_train)
past_50_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=30, max_samples=0.5, bootstrap=False, random_state=42).fit(X_train, y_train)
rnd_clf = RandomForestClassifier(n_estimators=30, random_state=42).fit(X_train, y_train)
ada_clf = AdaBoostClassifier(n_estimators=30).fit(X_train, y_train)
grad_clf = GradientBoostingClassifier(n_estimators=30, random_state=42).fit(X_train, y_train)

/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when 

In [13]:
bag_predicted_train = bag_clf.predict(X_train)
bag_predicted_test = bag_clf.predict(X_test)

bag_50_predicted_train = bag_50_clf.predict(X_train)
bag_50_predicted_test = bag_50_clf.predict(X_test)

past_predicted_train = past_clf.predict(X_train)
past_predicted_test = past_clf.predict(X_test)

past_50_predicted_train = past_50_clf.predict(X_train)
past_50_predicted_test = past_50_clf.predict(X_test)

rnd_predicted_train = rnd_clf.predict(X_train)
rnd_predicted_test = rnd_clf.predict(X_test)

ada_predicted_train = ada_clf.predict(X_train)
ada_predicted_test = ada_clf.predict(X_test)

grad_predicted_train = grad_clf.predict(X_train)
grad_predicted_test = grad_clf.predict(X_test)

In [14]:
acc_list2 = []

acc_list2.append((accuracy_score(y_train, bag_predicted_train), accuracy_score(y_test, bag_predicted_test)))
acc_list2.append((accuracy_score(y_train, bag_50_predicted_train), accuracy_score(y_test, bag_50_predicted_test)))
acc_list2.append((accuracy_score(y_train, past_predicted_train), accuracy_score(y_test, past_predicted_test)))
acc_list2.append((accuracy_score(y_train, past_50_predicted_train), accuracy_score(y_test, past_50_predicted_test)))
acc_list2.append((accuracy_score(y_train, rnd_predicted_train), accuracy_score(y_test, rnd_predicted_test)))
acc_list2.append((accuracy_score(y_train, ada_predicted_train), accuracy_score(y_test, ada_predicted_test)))
acc_list2.append((accuracy_score(y_train, grad_predicted_train), accuracy_score(y_test, grad_predicted_test)))
acc_list2

[(0.9956043956043956, 0.6754385964912281),
 (0.9296703296703297, 0.6842105263157895),
 (1.0, 0.6228070175438597),
 (0.9736263736263736, 0.6491228070175439),
 (0.9956043956043956, 0.6754385964912281),
 (0.8, 0.7368421052631579),
 (0.8373626373626374, 0.7105263157894737)]

In [15]:
with open('acc_bag.pkl', 'wb') as acc_bag_pickle:
    pickle.dump(acc_list2, acc_bag_pickle)

In [16]:
clf_list2 = []
clf_list.append(bag_clf)
clf_list.append(bag_50_clf)
clf_list.append(past_clf)
clf_list.append(past_50_clf)
clf_list.append(rnd_clf)
clf_list.append(ada_clf)
clf_list.append(grad_clf)

In [17]:
with open('bag.pkl', 'wb') as bag_pickle:
    pickle.dump(clf_list2, bag_pickle)

In [18]:
cancer_X_all_features = pd.DataFrame(data=data_breast_cancer['data'], columns=data_breast_cancer['feature_names'])
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(cancer_X_all_features, cancer_y, test_size=0.2, random_state=42)

In [19]:
sampling = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30,
                             bootstrap=True, bootstrap_features=False, max_features=2, 
                             max_samples=0.5, random_state=42).fit(X_train_all, y_train_all)

/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
sampling_predicted_train = sampling.predict(X_train_all)
sampling_predicted_test = sampling.predict(X_test_all)

In [21]:
acc_list3 = []
acc_list3.append(accuracy_score(y_train_all, sampling_predicted_train))
acc_list3.append(accuracy_score(y_test_all, sampling_predicted_test))
acc_list3

[0.9912087912087912, 0.9736842105263158]

In [22]:
with open('acc_fea.pkl', 'wb') as acc_fea_pickle:
    pickle.dump(acc_list3, acc_fea_pickle)

In [23]:
fea_list = [sampling]
with open('fea.pkl', 'wb') as fea_pickle:
    pickle.dump(fea_list, fea_pickle)

In [24]:
estimators_df = pd.DataFrame()
i = 0
for estimator in sampling.estimators_:
    features = sampling.estimators_features_[i]
    estimator_predicted_train = estimator.predict(X_train_all.iloc[:, features])
    estimator_predicted_test = estimator.predict(X_test_all.iloc[:, features])
    new_row = pd.DataFrame([accuracy_score(y_train_all, estimator_predicted_train), 
                            accuracy_score(y_test_all, estimator_predicted_test), 
                            [name for name in cancer_X_all_features.columns[features]]]).T
    estimators_df = estimators_df.append(new_row)
    i += 1
estimators_df.columns=['Train_acc', 'Test_acc', 'Feature_names']
estimators_df

/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/var/folders/p3/1mlnmr7563b2lsdz5xhg4k7w0000gn/T/ipykernel_95803/3596336254.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  estimators_df = estimators_df.append(new_row)
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/eryk/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without

,Train_acc,Test_acc,Feature_names
0,0.830769,0.745614,"[worst compactness, concave points error]"
0,0.916484,0.877193,"[mean radius, mean concavity]"
0,0.8,0.666667,"[concavity error, fractal dimension error]"
0,0.804396,0.631579,"[mean fractal dimension, concavity error]"
0,0.931868,0.903509,"[worst area, mean fractal dimension]"
0,0.872527,0.894737,"[mean radius, perimeter error]"
0,0.938462,0.921053,"[worst perimeter, mean smoothness]"
0,0.832967,0.850877,"[concave points error, area error]"
0,0.830769,0.72807,"[mean texture, worst compactness]"
0,0.92967,0.938596,"[mean area, worst radius]"


In [25]:
estimators_df = estimators_df.sort_values(['Test_acc', 'Train_acc'], ascending = [False, False])
estimators_df

,Train_acc,Test_acc,Feature_names
0,0.92967,0.938596,"[mean area, worst radius]"
0,0.938462,0.921053,"[worst perimeter, mean smoothness]"
0,0.934066,0.921053,"[worst concave points, concave points error]"
0,0.925275,0.921053,"[perimeter error, worst radius]"
0,0.927473,0.912281,"[worst radius, area error]"
0,0.931868,0.903509,"[worst area, mean fractal dimension]"
0,0.927473,0.903509,"[worst perimeter, symmetry error]"
0,0.872527,0.894737,"[mean radius, perimeter error]"
0,0.918681,0.877193,"[mean radius, mean compactness]"
0,0.916484,0.877193,"[mean radius, mean concavity]"


In [26]:
with open('acc_fea_rank.pkl', 'wb') as fea_rank:
    pickle.dump(estimators_df, fea_rank)